In [ ]:
# Importação das bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
PATH = '/content/drive/My Drive/TCCDataScience/datasets/mlsmall/'
movies = pd.read_csv(PATH+'movies.csv')
#genome_scores = pd.read_csv(PATH+'genome-scores.csv')
tags = pd.read_csv(PATH+'tags.csv')
#genome_tags = pd.read_csv(PATH+'genome-tags.csv')
#Use ratings data to downsample tags data to only movies with ratings 
ratings = pd.read_csv(PATH+'ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#AQUI IREMOS TRANSFORMAR A COLUNA GENEROS EM VARIAS COLUNAS BOOLEANAS
#PRIMEIRO CRIA-SE UM NOVO DATAFRAME A PARTIR DOS GENEROS DOS FILMES
generos = movies.genres.str.get_dummies()

#CONCATENA OS DOIS DATAFRAMES
dados_dos_filmes = pd.concat([movies, generos], axis=1)
dados_dos_filmes.head()

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dados_dos_filmes.drop(['genres'], axis=1, inplace=True)

In [ ]:
ratings_f = ratings.groupby('userId').filter(lambda x: len(x) >= 55) 

In [ ]:
# list the movie titles that survive the filtering
movie_list_rating = ratings_f.movieId.unique().tolist() 

In [ ]:
# no worries: we have kept 97% of the original movie titles in ratings data frame
len(ratings_f.movieId.unique())/len(dados_dos_filmes.movieId.unique()) * 100 

98.7990145760624

In [ ]:
 # but only 57% of the users 
len(ratings_f.userId.unique())/len(ratings.userId.unique()) * 100

59.67213114754099

In [ ]:
filmes = dados_dos_filmes[dados_dos_filmes.movieId.isin(movie_list_rating)]

In [ ]:
tags.drop(['timestamp'],1, inplace=True)
ratings_f.drop(['timestamp'],1, inplace=True)

KeyError: ignored

In [ ]:
ratings_f1 = pd.merge(filmes, ratings_f, on="movieId", how="right")

In [ ]:
ratings_f1.head(3)

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4.0,964982703
1,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,4.5,1106635946
2,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,2.5,1510577970


In [ ]:
def genClass(input):
  if(input):
    if(input >= 0. and input <2.5):
      return -1
    if(input >= 2.5 and input <= 3.5):
      return 0
    if(input > 3.5 and input <= 5.0):
      return 1

In [ ]:
ratings_f1['rating_c'] = ratings_f1['rating'].apply(genClass)

In [ ]:
ratings_f1.head(3)


,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp,rating_c
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4.0,964982703,1
1,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,4.5,1106635946,1
2,1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,2.5,1510577970,0


In [ ]:
ratings_f1.iloc[0:, 24:].head(3)

""
0
1
2


In [ ]:
previsores = ratings_f1.iloc[0:, 0:23]
classe = ratings_f1.iloc[0:, 24:]

In [ ]:
previsores.info()
previsores.drop(['title'], axis=1, inplace=True)
previsores.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92729 entries, 0 to 92728
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             92729 non-null  int64 
 1   title               92729 non-null  object
 2   (no genres listed)  92729 non-null  int64 
 3   Action              92729 non-null  int64 
 4   Adventure           92729 non-null  int64 
 5   Animation           92729 non-null  int64 
 6   Children            92729 non-null  int64 
 7   Comedy              92729 non-null  int64 
 8   Crime               92729 non-null  int64 
 9   Documentary         92729 non-null  int64 
 10  Drama               92729 non-null  int64 
 11  Fantasy             92729 non-null  int64 
 12  Film-Noir           92729 non-null  int64 
 13  Horror              92729 non-null  int64 
 14  IMAX                92729 non-null  int64 
 15  Musical             92729 non-null  int64 
 16  Mystery             92

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId
0,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7
2,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15
3,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17
4,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18


In [ ]:
previsores.fillna(0,inplace=True)

In [ ]:
classe.shape 
classe.shape
classe.fillna(0)

,timestamp,rating_c
0,964982703,1
1,1106635946,1
2,1510577970,0
3,1305696483,1
4,1455209816,0
...,...,...
92724,1537109082,1
92725,1537109545,0
92726,1537109805,0
92727,1537110021,0


In [ ]:
# Transformação da classe para o formato "dummy", pois temos uma rede neural com 3 neurônios na camada de saída
classe_dummy = np_utils.to_categorical(classe)
classe_dummy

MemoryError: ignored

In [ ]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)